In [1]:
import boto3
import numpy as np
from langchain_community.vectorstores import Chroma
import shutil
import sys
import os
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.document_loaders import PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.embeddings.bedrock import BedrockEmbeddings

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import Chroma

from botocore.exceptions import ClientError
from langchain.prompts import ChatPromptTemplate

import json

import psycopg2
import warnings
import pickle

from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")

In [2]:
DATABASE_PATH = os.environ.get("DATABASE_PATH", "../data/database")
DATA_PATH = os.environ.get("DATABASE_PATH", "../data/docs")

In [3]:
import glob
import os

# Path to the directory
directory_path = '../data/docs'

# Find all PDF files recursively
pdf_files = glob.glob(os.path.join(directory_path, '**', '*.pdf'), recursive=True)

In [21]:
def cosine_similarity(a, b):
    """
    Compute the cosine similarity between two vectors.
    
    Parameters:
    a (numpy.ndarray): First vector.
    b (numpy.ndarray): Second vector.
    
    Returns:
    float: Cosine similarity between the two vectors.
    """
    # Compute the dot product of the two vectors
    dot_product = np.dot(a, b)
    
    # Compute the norms (magnitudes) of the vectors
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    # Avoid division by zero
    if norm_a == 0 or norm_b == 0:
        return 0.0
    
    # Compute cosine similarity
    return dot_product / (norm_a * norm_b)



In [77]:
def query_llm(conversation, client, model_id, max_tokens):
    try:
        # Send the message to the model, using a basic inference configuration
        response = client.converse(
                    modelId=model_id,
                    messages=conversation,
                    inferenceConfig={"maxTokens": max_tokens, "temperature": 1},
                    additionalModelRequestFields={"top_k": 250, "top_p": 1},
        )

        # Extract and print the response text
        return response["output"]["message"]["content"][0]["text"]
        #print(response_text)

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

def extract_name_year(splits, client, model_id, n_page = 3):

    doc_identifier = [*splits[:n_page], *splits[-n_page:]]
    doc_identifier = "\n\n---\n\n".join([doc.page_content for doc in doc_identifier])

    prompt_text = f"""

    Question : What is the name of the company, what financial year this report is about?

    Output : only json string format with two keys (company, year)

    Error : if you could not find anything the values in json should be empty strings

    Context : {doc_identifier}

    """

    conversation = [
            {
                "role": "user",
                "content": [{"text": prompt_text}]  # Wrap the prompt in a list of dictionaries
            }
        ]

    response = query_llm(conversation, client, model_id)
    return response

def add_name_year_tags(split_documents, info_dict):
    embedder = BedrockEmbeddings()
    modified_docs = []
    for doc in tqdm(split_documents):
        if info_dict['company'] and info_dict['year']:
            modified_content = f"""
                <company> {info_dict['company']} <company>
                <year> {info_dict['company']} <year>

                {doc.page_content}

                <company> {info_dict['company']} <company>
                <year> {info_dict['company']} <year>
                """
        else:
            modified_content = doc.page_content

        modified_docs.append(dict(company = info_dict['company'], year = info_dict['year'], embedding = embedder.embed_query(modified_content), content = modified_content))

    return modified_docs

In [45]:
model_id = "anthropic.claude-3-haiku-20240307-v1:0"
client = boto3.client("bedrock-runtime", region_name="us-west-2")
embedder = BedrockEmbeddings()

In [17]:
pdf_path = pdf_files[5]

loader = PyPDFLoader(pdf_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=5000)
split_documents = text_splitter.split_documents(documents)

info_dict = extract_name_year(split_documents, client, model_id, n_page = 3)

info_dict = json.loads(info_dict)

docs = add_name_year_tags(split_documents, info_dict)

  0%|          | 0/218 [00:00<?, ?it/s]

In [86]:
with open("../data/database/db.pkl", "rb") as file:
    docs = pickle.load(file)

set([(d['company'], d['year']) for d in docs])

{('ALTAGAS', '2022'),
 ('Alimentation Couche-Tard', '2022'),
 ('Alimentation Couche-Tard Inc.', '2018'),
 ('Alimentation Couche-Tard Inc.', '2019'),
 ('Alimentation Couche-Tard Inc.', '2024'),
 ('Alimentation Couche-Tard inc.', '2020'),
 ('Alimentation Couche-Tard inc.', '2021'),
 ('Alimentation Couche-Tard inc.', '2023'),
 ('AltaGas Ltd.', '2020'),
 ('AltaGas Ltd.', '2021'),
 ('AltaGas Ltd.', '2023'),
 ('BCE Inc.', '2018'),
 ('BCE Inc.', '2019'),
 ('BCE Inc.', '2020'),
 ('BCE Inc.', '2021'),
 ('BCE Inc.', '2022'),
 ('BCE Inc.', '2023'),
 ('COGECO COMMUNICATIONS INC.', '2020'),
 ('CP', '2020'),
 ('Canadian Pacific', '2018'),
 ('Canadian Pacific', '2021'),
 ('Canadian Pacific', '2022'),
 ('Canadian Pacific Kansas City Limited', '2023'),
 ('Canadian Pacific Railway Limited', '2019'),
 ('CogeCo CommuniCations inC', '2018'),
 ('Cogeco Communications Inc.', '2019'),
 ('Cogeco Communications Inc.', '2021'),
 ('Cogeco Communications Inc.', '2022'),
 ('Cogeco Communications inc.', '2023'),
 ('

In [91]:
documents = [d for d in docs if d['company'] == 'Fortis Inc.' and d['year'] == '2021']
len(documents)

89

In [96]:
query_chunk = """
basic financial indicators

Turnover, gross margin, free cash flow, net debt, profit (before interest, taxes, depreciation and amortization), earnings per share

"""

embedding_query_chunk = embedder.embed_query(query_chunk)

sim = [cosine_similarity(embedding_query_chunk, d['embedding']) for d in documents]

relevant_docs = "\n\n ---- \n\n".join([documents[i]['content'] for i in np.argsort(sim)[-50:][::-1]])


prompt_text = f"""

    RESPONSE LANGUAGE : please response in english

    TASK : retrieve basic financial indicators

    VALUES : Turnover, gross margin, free cash flow, net debt, profit (before interest, taxes, depreciation and amortization), earnings per share

    OUTPUT FORMAT : put everything in a table. how the values has changed comparing to previous years
    
    indicate the unit of the values and put NA for values that are not available

    Context : {relevant_docs}
    """

conversation = [{
            "role": "user",
            "content": [{"text": prompt_text}]  # Wrap the prompt in a list of dictionaries
    }]

response_llm = query_llm(conversation, client, model_id, max_tokens = 4096)

In [97]:
print(response_llm)

Here is the table with the financial indicators you requested, along with how the values have changed compared to previous years and the unit of the values. I have indicated NA for values that are not available.

Financial Indicator	2021	Variance compared to previous year
Turnover	$9,448 million	Increased by $513 million compared to 2020
Gross margin	NA	NA
Free cash flow	$2,907 million	Increased by $206 million compared to 2020
Net debt	$25,784 million	Increased by $1,203 million compared to 2020
Profit (before interest, taxes, depreciation and amortization)	$2,469 million	Decreased by $39 million compared to 2020
Earnings per share	$2.61	Increased by $0.01 compared to 2020


In [94]:


# Paths and settings
# pdf_path = "../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf"
# database_path = "../data/database/db.sqlite3"

# Load and split the document
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_documents = text_splitter.split_documents(documents)

# Initialize Bedrock embeddings
embedding_model = BedrockEmbeddings()

# Initialize Chroma database
database = Chroma(
    persist_directory=database_path,
    embedding_function=BedrockEmbeddings()
)

# Process each document chunk
for doc in split_documents:
    # Generate the embedding for the chunk's content
    doc_embedding = embedding_model.embed_query(doc.page_content)
    
    # Update the document metadata with the embedding if needed
    doc.metadata["embedding"] = doc_embedding

    # Add document with embedding to the database
    database.add_documents([doc])

# Persist the database
database.persist()

print("Document chunks and embeddings stored successfully.")


ValueError: Expected metadata value to be a str, int, float or bool, got [0.86328125, 0.310546875, -0.0947265625, -0.00872802734375, 0.7890625, 0.390625, -0.03076171875, -0.0004730224609375, -0.001983642578125, -1.0703125, -0.2294921875, 0.0306396484375, -0.40625, 0.45703125, -0.796875, -0.107421875, 0.283203125, 0.439453125, -0.36328125, -0.396484375, -0.7578125, -0.03369140625, 0.671875, -0.58984375, 0.228515625, 0.60546875, 0.6875, -0.60546875, -0.201171875, -0.52734375, -0.0693359375, -0.423828125, 0.21875, -0.52734375, 0.049560546875, 0.058837890625, -0.08154296875, 0.337890625, -0.0869140625, -0.71875, -0.9296875, 0.0245361328125, 0.400390625, -0.201171875, -0.28125, 0.0036773681640625, -0.2265625, -0.37890625, 0.8125, -0.232421875, -0.23046875, -0.126953125, -0.203125, 0.279296875, 0.2158203125, -0.84765625, 0.39453125, 0.02001953125, 0.40234375, -0.27734375, 0.3671875, 0.43359375, 0.349609375, 0.466796875, 0.31640625, 0.0206298828125, -0.22265625, -0.15625, 0.026123046875, 0.42

In [19]:
# Embedding function using Bedrock embeddings
def get_embedding_function():
    embeddings = BedrockEmbeddings()
    return embeddings

# Function to split documents into chunks
def split_documents(documents: list[Document]):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=10000,
        chunk_overlap=5000,
        length_function=len
    )
    return splitter.split_documents(documents)

# Function to calculate unique chunk IDs for documents
def calculate_chunk_ids(chunks):
    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the chunk metadata
        chunk.metadata["id"] = chunk_id

    return chunks

# Function to generate the database from the documents
def generate_database(path):
    # Load the documents
    # loader = PyPDFDirectoryLoader(path)
    loader = PyPDFLoader(path)
    documents = loader.load()
    
    # Split the documents into chunks
    documents = split_documents(documents)
    
    # Initialize the Chroma database
    database = Chroma(
        persist_directory=DATABASE_PATH,
        embedding_function=get_embedding_function()
    )
    
    # Calculate IDs for chunks
    chunks_with_ids = calculate_chunk_ids(documents)
    
    existing_items = database.get(include=[])  # Retrieve existing document IDs
    existing_ids = set(existing_items["ids"])
    
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # # Add only new documents to the database
    new_chunks = []
    for chunk in chunks_with_ids:
         if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)
    return new_chunks
    if new_chunks:
        print(f"Adding new documents to the database: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]

        database.add_documents(new_chunks, ids=new_chunk_ids)

        database.persist()
    else:
        print("All the documents are added")
    
    return documents

# Run the function
# generate_database()

In [83]:
report_path = "../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf"
new_chunks = generate_database(report_path)

Number of existing documents in DB: 282


In [91]:
d = new_chunks[22]
type(d)

langchain_core.documents.base.Document

In [85]:
new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
new_chunk_ids

['../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf:0:0',
 '../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf:1:0',
 '../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf:2:0',
 '../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf:3:0',
 '../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf:4:0',
 '../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf:5:0',
 '../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf:6:0',
 '../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf:7:0',
 '../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf:8:0',
 '../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf:9:0',
 '../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf:10:0',
 '../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf:11:0',
 '../data/docs/Consommation de Base/Métro/2021_Rapport annuel_FR.pdf:12:0',
 '../data/docs/Consomm

In [78]:
loader = PyPDFLoader(report_path)
documents = loader.load()

# Split the documents into chunks
documents = split_documents(documents)
documents

[Document(metadata={'source': '../data/docs/Consommation de Base/Métro/2020-annuel-10Q-FR-FINAL.pdf', 'page': 0}, page_content='Rapport annuel\n2020'),
 Document(metadata={'source': '../data/docs/Consommation de Base/Métro/2020-annuel-10Q-FR-FINAL.pdf', 'page': 1}, page_content="PROFIL CORPORATIF\nMETRO INC. est un chef de file dans les domaines de l'alimentation et de la pharmacie au Québec et en Ontario. Par \nses activités de détaillant, franchiseur, distributeur et fabricant, elle exploite ou approvisionne un réseau de 953 \nmagasins d’alimentation sous plusieurs bannières dont Metro, Metro Plus, Super C, Food Basics, Adonis et Première \nMoisson, de même que 648 pharmacies principalement sous les bannières Jean Coutu, Brunet, Metro Pharmacy et \nFood Basics Pharmacy, qui procurent de l'emploi directement ou indirectement à près de 90 000 personnes.\nFAITS SAILLANTS 2020 \n• Chiffre d'affaires de 17 997,5 millions $, en hausse de 7,3 % et de 7,7 % en excluant l'impact de l'IFRS 16\

In [79]:
len(documents)

95

In [20]:
def get_embedding_function():
    return BedrockEmbeddings()

def get_relevant_context(query, prompt, k = 5):
    embedding_function_instance = get_embedding_function()
    db = Chroma(persist_directory=DATABASE_PATH, embedding_function=embedding_function_instance)

    results = db.similarity_search_with_score(query, k = 5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(prompt)

    prompt_text = prompt_template.format(context=context_text, question=query)

    conversation = [
        {
            "role": "user",
            "content": [{"text": prompt_text}]  # Wrap the prompt in a list of dictionaries
        }
    ]

    return conversation, results

def query_llm(conversation, client, model_id):
    try:
        # Send the message to the model, using a basic inference configuration
        response = client.converse(
                    modelId=model_id,
                    messages=conversation,
                    inferenceConfig={"maxTokens": 4000, "temperature": 1},
                    additionalModelRequestFields={"top_k": 250, "top_p": 1},
        )

        # Extract and print the response text
        return response["output"]["message"]["content"][0]["text"]
        #print(response_text)

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

In [58]:
model_id = "anthropic.claude-3-haiku-20240307-v1:0"
client = boto3.client("bedrock-runtime", region_name="us-west-2")

In [76]:
query_text = """
    Basic financial indicators: Turnover, gross margin, free cash flow, net debt, profit (before interest,
    taxes, depreciation and amortization), earnings per share, for Metro company in 2023 metro metro metro
"""

PROMPT_TEMPLATE = """
                    <meta>
                        current year: 2024
                        role: financial analyst
                        language: english
                        expertise: finance, annual reports, financial performance
                        tone: professional, concise, objective
                        response style: factual, summary-focused
                    <meta>

                    You are a financial analyst. Answer the following question in English, using only the information from the provided context.

                    Question: {question}

                    Context:  {context}

                    Output: Response should be a table with all the required variables as rows, different years as columns, variation as column, leave it empty if the value does not exist.

                    Answer in a clear and concise manner, focusing on key financial insights.
                    """

In [77]:
# Format the sources and print
conversation, relevant_results = get_relevant_context(query=query_text, prompt=PROMPT_TEMPLATE, k = 5)

response_text = query_llm(conversation, client, model_id)

sources = [doc.metadata.get("id", None) for doc, _score in relevant_results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response: Here is a table with the key financial indicators for Metro company in 2023:

| Metric | 2023 |
|--------|------|
| Turnover | - |
| Gross margin | - |
| Free cash flow | - |
| Net debt | - |
| Profit (before interest, taxes, depreciation and amortization) | - |
| Earnings per share | - |

Based on the provided context, the specific financial results for Metro company in 2023 were not disclosed. The information given focused on Metro's share buyback program, dividend payments, and stock performance, but did not provide the detailed financial metrics requested in the question. Therefore, I am unable to populate the table with the specific 2023 values for Metro. The context indicates Metro had a successful year, with increases in revenue, net income, and earnings per share, but the exact figures were not provided.
Sources: ['../data/docs/Consommation de Base/Métro/2020-annuel-10Q-FR-FINAL.pdf:25:0', '../data/docs/Consommation de Base/Métro/2020-annuel-10Q-FR-FINAL.pdf:13:0', '.

In [62]:
# Format the sources and print
conversation, relevant_results = get_relevant_context(query=query_text, prompt=PROMPT_TEMPLATE, k = 5)

response_text = query_llm(conversation, client, model_id)

sources = [doc.metadata.get("id", None) for doc, _score in relevant_results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response: Here is the requested financial information for ROGERS company in a table format:

| Metric                           | 2023 | 2022 | Variation |
|----------------------------------|------|------|------------|
| Turnover                         | 19,308 | 15,396 | 25% |
| Gross Margin                    | N/A | N/A | N/A |
| Free Cash Flow                   | 2,414 | 1,773 | 36% |
| Net Debt                         | N/A | N/A | N/A |
| Profit (EBITDA)                  | 8,581 | 6,393 | 34% |
| Earnings per Share               | 4.60 | 3.79 | 21% |

Key Insights:
- Turnover increased by 25% in 2023 compared to 2022, driven by growth in both service revenue (+27%) and equipment revenue (+17%).
- Free cash flow grew by 36%, reaching $2,414 million, reflecting the strong operating performance.
- EBITDA increased by 34% to $8,581 million, with the EBITDA margin expanding by 2.9 percentage points to 44.4%.
- Adjusted earnings per share rose by 21% to $4.60, showcasing the company'

In [13]:



# # Query and generate context for the prompt
# query_text = "Combien Bell Canada a tiré un produit brut totalisant à la suite de l'émission de débentures à moyen terme (MTN) de 7 ans et de 10 ans ?"


# results = db.similarity_search_with_score(query_text, k=5)

# context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
# prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
# prompt = prompt_template.format(context=context_text, question=query_text)

# # Prepare the conversation payload for Bedrock
# conversation = [
#     {
#         "role": "user",
#         "content": [{"text": prompt}]  # Wrap the prompt in a list of dictionaries
#     }
# ]

# Set up the model client
model_id = "anthropic.claude-3-haiku-20240307-v1:0"
client = boto3.client("bedrock-runtime", region_name="us-west-2")

try:
    # Send the message to the model, using a basic inference configuration
    response = client.converse(
                modelId=model_id,
                messages=conversation,
                inferenceConfig={"maxTokens": 4000, "temperature": 1},
                additionalModelRequestFields={"top_k": 250, "top_p": 1},
    )

    # Extract and print the response text
    response_text = response["output"]["message"]["content"][0]["text"]
    #print(response_text)

    # Format the sources and print
    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

Response: Selon les informations fournies dans le rapport annuel 2019 de BCE Inc., Bell Canada a émis avec succès des débentures à moyen terme (MTN) d'une durée de 7 ans et de 10 ans, totalisant un produit brut de 1,15 milliard de dollars canadiens.

Spécifiquement:

- Le 10 septembre 2019, Bell Canada a émis des MTN, série M-50, à 2,90%, d'un capital de 550 millions de dollars, échéant le 10 septembre 2029. 

- Le 13 mai 2019, Bell Canada a émis des MTN, série M-49, à 2,75%, d'un capital de 600 millions de dollars, échéant le 29 janvier 2025.

Donc, le produit brut total tiré de l'émission de ces deux séries de débentures MTN de 7 ans et 10 ans s'élève à 1,15 milliard de dollars canadiens.
Sources: ['data/docs/2019-rapport-annuel-bce.pdf:146:0', 'data/docs/2018-rapport-annuel-bce.pdf:157:0', 'data/docs/2019-rapport-annuel-bce.pdf:36:0', 'data/docs/2018-rapport-annuel-bce.pdf:155:0', 'data/docs/2018-rapport-annuel-bce.pdf:38:0']


In [ ]:
["give me turnover based on the report", "give earnings per share based on the report", ...]